# CLTV Prediction

### Variables

* **'GuestID'**: Identification Number of Guest
* **'Status'**: Status of Booking
* **'RoomGroupID'**: Identification Number of Room Group
* **'CreatedDate'**: Date of Creating Booking
* **'ArrivalDate'**: Date of Arrival
* **'DepartureDate'**: Date of Departure
* **'RoomPrice'**: Price of Room per Day
* **'Channel'**: Channel of Booking
* **'RoomNo'**: Identification Number of Room
* **'Country'**: Country of Guest
* **'Adults'**: Number of Adults
* **'Children'**: Number of Children
* **'TotalPayment'**: Total Fee Paid

### Import libraries and set up some cosmetic settings

In [1]:
!pip install lifetimes

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

### Read and copy the dataset

In [3]:
df_ = pd.read_csv("/Users/hikmetburakozcan/Exercises/Datasets/bookings.csv")
df = df_.copy()

### A quick look at the dataset

In [4]:
df.shape

(10000, 13)

In [5]:
df.head()

,GuestID,Status,RoomGroupID,CreatedDate,ArrivalDate,DepartureDate,RoomPrice,Channel,RoomNo,Country,Adults,Children,TotalPayment
0,53801,O,1,2017-12-19,2018-09-09,2018-09-19,230.8900,NaN,467.0000,au,3,0,2309.9300
1,37129,O,1,2017-12-20,2018-01-03,2018-01-05,170.1500,other,467.0000,us,0,0,340.9400
2,37708,O,1,2017-12-20,2018-01-17,2018-01-20,176.8100,NaN,647.0000,jp,0,0,542.5900
3,41515,O,1,2017-12-20,2018-02-18,2018-02-23,226.8000,NaN,508.0000,gb,1,0,1134.2900
4,37390,O,1,2017-12-21,2018-01-11,2018-01-12,129.8000,offline,529.0000,cn,1,0,140.5900


In [6]:
df.isnull().sum()

GuestID             0
Status              0
RoomGroupID         0
CreatedDate         0
ArrivalDate         0
DepartureDate       0
RoomPrice           0
Channel          3719
RoomNo            210
Country           482
Adults              0
Children            0
TotalPayment        0
dtype: int64

In [7]:
df.describe([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
GuestID,10000.0000,93701.7030,57906.2559,1.0000,2099.9400,14949.5000,46657.5000,68300.0000,133578.0000,198799.0000,207096.2000,209312.0000
RoomGroupID,10000.0000,1.2135,0.4256,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.0000,2.0000,3.0000
RoomPrice,10000.0000,205.2475,52.2294,0.0000,54.5779,111.7490,176.9800,208.6050,234.9300,284.8510,324.6219,507.2600
RoomNo,9790.0000,507.3632,167.3369,202.0000,207.0000,233.0000,358.0000,523.0000,651.0000,751.0000,767.0000,770.0000
Adults,10000.0000,1.4460,0.9707,0.0000,0.0000,0.0000,1.0000,1.0000,2.0000,3.0000,3.0000,4.0000
Children,10000.0000,0.0229,0.1720,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.0000
TotalPayment,10000.0000,627.6992,642.9681,1.4300,79.5585,147.0180,220.7425,383.5700,848.6975,1747.6870,3120.8267,8616.7100


In [8]:
df.describe(include="object").T

,count,unique,top,freq
Status,10000,4,O,9524
CreatedDate,10000,755,2018-04-09,25
ArrivalDate,10000,888,2019-05-10,24
DepartureDate,10000,886,2018-06-22,28
Channel,6281,3,online,2743
Country,9518,114,cn,1379


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GuestID        10000 non-null  int64  
 1   Status         10000 non-null  object 
 2   RoomGroupID    10000 non-null  int64  
 3   CreatedDate    10000 non-null  object 
 4   ArrivalDate    10000 non-null  object 
 5   DepartureDate  10000 non-null  object 
 6   RoomPrice      10000 non-null  float64
 7   Channel        6281 non-null   object 
 8   RoomNo         9790 non-null   float64
 9   Country        9518 non-null   object 
 10  Adults         10000 non-null  int64  
 11  Children       10000 non-null  int64  
 12  TotalPayment   10000 non-null  float64
dtypes: float64(3), int64(4), object(6)
memory usage: 1015.8+ KB


### Some Data Manupilations

In [10]:
df = df[(~((df["Adults"] == 0) & (df["Children"] == 0))) & (df["RoomPrice"] > 0)]

In [11]:
df["Channel"].fillna("other", inplace=True)

In [12]:
df["Country"].fillna("otr", inplace=True)

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

GuestID          0
Status           0
RoomGroupID      0
CreatedDate      0
ArrivalDate      0
DepartureDate    0
RoomPrice        0
Channel          0
RoomNo           0
Country          0
Adults           0
Children         0
TotalPayment     0
dtype: int64

In [15]:
df.shape

(7934, 13)

**10000 - 7934 = 2066 observation units are dropped**

### Outliers

In [16]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.05)
    quartile3 = dataframe[variable].quantile(0.95)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [17]:
outlier_thresholds(df, "TotalPayment")

(-2167.7332499999998, 3992.3687499999996)

In [18]:
outlier_thresholds(df, "RoomPrice")

(-154.14750000000004, 545.6685)

In [19]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [20]:
replace_with_thresholds(df, "TotalPayment")

In [21]:
replace_with_thresholds(df, "RoomPrice")

In [22]:
df.describe([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
GuestID,7934.0000,93658.5006,57459.1232,1.0000,2058.4900,13910.0000,46786.0000,69138.5000,133588.0000,197219.5000,205131.0400,209221.0000
RoomGroupID,7934.0000,1.2342,0.4402,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.0000,2.0000,3.0000
RoomPrice,7934.0000,202.8905,52.4914,1.4500,56.0757,108.2835,174.0625,206.5150,233.2700,283.2375,321.5369,507.2600
RoomNo,7934.0000,505.6276,168.7558,202.0000,207.0000,231.0000,356.0000,520.0000,650.0000,753.0000,767.0000,770.0000
Adults,7934.0000,1.7710,0.7536,0.0000,1.0000,1.0000,1.0000,2.0000,2.0000,3.0000,3.0000,4.0000
Children,7934.0000,0.0285,0.1917,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.0000
TotalPayment,7934.0000,607.9064,583.9317,5.2400,81.1156,142.3050,218.1425,373.6850,835.9550,1682.3305,2964.0386,3992.3687


### CLTV Prediction

In [23]:
df["DepartureDate"].max()

'2020-09-26'

In [24]:
analysis_date = dt.datetime(2020, 9, 28)

In [25]:
for col in df.columns[df.columns.str.contains("Date")]:
    df[col] = pd.to_datetime(df[col])

* **recency**: Time since last booking. Weekly. 
* **T**: Customer's age. Weekly. (how long before the analysis date the first booking was made)
* **frequency**: total number of repeat booking (frequency>1)
* **monetary**: average earnings per booking

In [26]:
cltv_df = df.groupby('GuestID').agg({'ArrivalDate': ['nunique', 'min', lambda x: (analysis_date - x.min()).days],
                                     'TotalPayment': "sum",
                                     'DepartureDate': 'max'})

In [27]:
cltv_df.head()

ArrivalDate                       TotalPayment DepartureDate
            nunique        min <lambda_0>          sum           max
GuestID                                                             
1                 1 2019-09-24        370    1337.8900    2019-09-29
37                2 2018-11-09        689     278.4400    2019-06-02
48                3 2018-02-16        955     766.6200    2019-05-04
156               2 2018-11-29        669    1398.6000    2019-08-10
191               2 2019-04-28        519     937.5500    2019-09-01

In [28]:
cltv_df.columns = cltv_df.columns.droplevel(0)

In [29]:
cltv_df.columns = ['frequency', 'First_Arrival_Date', 'T', 'monetary', 'Last_Departure_Date']

In [30]:
cltv_df.head()

,frequency,First_Arrival_Date,T,monetary,Last_Departure_Date
GuestID,,,,,
1,1,2019-09-24,370,1337.8900,2019-09-29
37,2,2018-11-09,689,278.4400,2019-06-02
48,3,2018-02-16,955,766.6200,2019-05-04
156,2,2018-11-29,669,1398.6000,2019-08-10
191,2,2019-04-28,519,937.5500,2019-09-01


In [31]:
cltv_df["recency"] = (cltv_df["Last_Departure_Date"] - cltv_df["First_Arrival_Date"]).dt.days

In [32]:
cltv_df = cltv_df[cltv_df["frequency"] > 1]

In [33]:
cltv_df["T_weekly"] = (cltv_df["T"])/7

In [34]:
cltv_df["recency_weekly"] = (cltv_df["T"])/7

In [35]:
cltv_df["monetary"] = cltv_df["monetary"]/cltv_df["frequency"]

In [36]:
cltv_df.head()

,frequency,First_Arrival_Date,T,monetary,Last_Departure_Date,recency,T_weekly,recency_weekly
GuestID,,,,,,,,
37,2,2018-11-09,689,139.2200,2019-06-02,205,98.4286,98.4286
48,3,2018-02-16,955,255.5400,2019-05-04,442,136.4286,136.4286
156,2,2018-11-29,669,699.3000,2019-08-10,254,95.5714,95.5714
191,2,2019-04-28,519,468.7750,2019-09-01,126,74.1429,74.1429
482,6,2019-04-11,536,699.1500,2019-08-28,139,76.5714,76.5714


In [37]:
cltv_df = cltv_df[["recency_weekly", "T_weekly", "frequency", "monetary"]]

In [38]:
cltv_df.head()

,recency_weekly,T_weekly,frequency,monetary
GuestID,,,,
37,98.4286,98.4286,2,139.2200
48,136.4286,136.4286,3,255.5400
156,95.5714,95.5714,2,699.3000
191,74.1429,74.1429,2,468.7750
482,76.5714,76.5714,6,699.1500


In [39]:
cltv_df.reset_index(inplace=True)

In [40]:
cltv_df.head()

,GuestID,recency_weekly,T_weekly,frequency,monetary
0,37,98.4286,98.4286,2,139.2200
1,48,136.4286,136.4286,3,255.5400
2,156,95.5714,95.5714,2,699.3000
3,191,74.1429,74.1429,2,468.7750
4,482,76.5714,76.5714,6,699.1500


### BG-NBD Model
* Predict expected number of transactions up to time

In [41]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [42]:
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 1350 subjects, a: 0.00, alpha: 284.37, b: 0.00, r: 6.73>

### Who are the 10 customers we expect to make booking the most in 3 months?

In [43]:
cltv_df["expected_booking_3_months"] = bgf.conditional_expected_number_of_purchases_up_to_time(4*3,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency_weekly'],
                                                        cltv_df['T_weekly'])

In [46]:
cltv_df.sort_values("expected_booking_3_months", ascending=False).head(10)

,GuestID,recency_weekly,T_weekly,frequency,monetary,expected_booking_3_months
358,44343,122.8571,122.8571,9,276.7767,0.4634
53,5620,78.8571,78.8571,7,916.6371,0.4535
4,482,76.5714,76.5714,6,699.1500,0.4231
431,48131,123.5714,123.5714,7,641.5429,0.4038
990,128350,66.8571,66.8571,5,341.7840,0.4006
174,30989,100.8571,100.8571,6,451.3000,0.3964
665,63552,96.1429,96.1429,5,443.6260,0.3698
89,11022,97.8571,97.8571,5,810.6880,0.3681
739,69493,65.8571,65.8571,4,421.4025,0.3675
613,59420,101.5714,101.5714,5,445.8900,0.3646


### GAMMA-GAMMA Model

In [47]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [48]:
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 1350 subjects, p: 3.66, q: 0.29, v: 3.56>

### Who are the 10 customers we expect to make the most profit

In [49]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

In [50]:
cltv_df.sort_values("expected_average_profit", ascending=False)

,GuestID,recency_weekly,T_weekly,frequency,monetary,expected_booking_3_months,expected_average_profit
529,55217,88.8571,88.8571,2,3992.3687,0.2806,4424.7376
187,34402,135.0000,135.0000,4,3905.8866,0.3069,4106.5909
427,47538,128.8571,128.8571,2,3691.5694,0.2534,4091.5105
250,39352,138.5714,138.5714,2,3627.2344,0.2476,4020.2399
337,43233,133.5714,133.5714,2,3561.3600,0.2505,3947.2639
...,...,...,...,...,...,...,...
1184,179084,103.2857,103.2857,2,93.0450,0.2701,105.0472
962,126504,94.2857,94.2857,2,92.9500,0.2765,104.9419
495,53041,116.4286,116.4286,2,87.8300,0.2613,99.2700
972,126947,90.0000,90.0000,2,81.4200,0.2797,92.1689


### Calculation of CLTV with BG-NBD and GG model

In [51]:
cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary'],
                                   time=3,  # 3 months
                                   freq="W", # frequency information of T.
                                   discount_rate=0.01)

In [52]:
cltv_df.head()

,GuestID,recency_weekly,T_weekly,frequency,monetary,expected_booking_3_months,expected_average_profit,cltv
0,37,98.4286,98.4286,2,139.2200,0.2736,156.2001,45.5009
1,48,136.4286,136.4286,3,255.5400,0.2774,274.5373,81.0875
2,156,95.5714,95.5714,2,699.3000,0.2756,776.6595,227.9411
3,191,74.1429,74.1429,2,468.7750,0.2921,521.2827,162.1351
4,482,76.5714,76.5714,6,699.1500,0.4231,723.2025,325.8395


### Creating Segments by CLTV

In [53]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

In [54]:
cltv_df.sort_values(by="cltv", ascending=False).head(10)

,GuestID,recency_weekly,T_weekly,frequency,monetary,expected_booking_3_months,expected_average_profit,cltv,segment
187,34402,135.0000,135.0000,4,3905.8866,0.3069,4106.5909,1342.1868,A
529,55217,88.8571,88.8571,2,3992.3687,0.2806,4424.7376,1321.9740,A
1295,198151,46.8571,46.8571,2,3065.3144,0.3161,3397.7421,1143.8593,A
427,47538,128.8571,128.8571,2,3691.5694,0.2534,4091.5105,1104.0883,A
250,39352,138.5714,138.5714,2,3627.2344,0.2476,4020.2399,1059.9388,A
337,43233,133.5714,133.5714,2,3561.3600,0.2505,3947.2639,1053.1488,A
386,45835,112.8571,112.8571,2,3228.6200,0.2636,3578.6529,1004.5915,A
692,65186,71.2857,71.2857,2,2854.3800,0.2944,3164.0679,992.0291,A
1043,133091,64.8571,64.8571,2,2774.1644,0.2998,3075.2047,981.9161,A
1047,133338,64.2857,64.2857,2,2581.8800,0.3003,2862.1911,915.3985,A


In [58]:
cltv_df.groupby("segment").agg({"recency_weekly": "mean",
                                "frequency": "mean",
                                "T_weekly": "mean",
                                "expected_booking_3_months": "mean",
                                "expected_average_profit": "mean",
                                "monetary": ["mean", "sum"]})

recency_weekly frequency T_weekly expected_booking_3_months expected_average_profit  monetary            
                  mean      mean     mean                      mean                    mean      mean         sum
segment                                                                                                          
D             100.3352    2.1036 100.3352                    0.2768                236.5512  212.5991  71858.4958
C              96.0360    2.2226  96.0360                    0.2839                427.9984  386.9786 130411.7973
B              95.3311    2.2819  95.3311                    0.2861                697.6580  632.3131 213089.5172
A              93.0499    2.2574  93.0499                    0.2872               1357.6711 1231.5076 416249.5648